In [1]:
#V2 will check if the path is empty in the csv. 
#If empty and available fill it,not empty not rewrite it
#v3 do it withoyt RT STRuctBuilder

In [2]:
#from rt_utils import RTStructBuilder
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import os
import csv
import pandas as pd

In [3]:
def FindCT_UID(path):
    files = os.listdir(path)
    ct_dicom_dataset = pydicom.dcmread(os.path.join(path,files[0]))
    series_instance_uid = ct_dicom_dataset[(0x0020, 0x0052)].value
    return series_instance_uid

In [4]:
def FindRT_UID_perContour(path,roi_name_to_find):
    files = os.listdir(path)
    rtstruct_dataset = pydicom.dcmread(path+files[0])
    if rtstruct_dataset.Modality == "RTSTRUCT":
        roi_sequence = rtstruct_dataset.StructureSetROISequence
        for roi_item in roi_sequence:
            roi_number = roi_item.ROINumber
            roi_name = roi_item.ROIName
            if roi_name == roi_name_to_find:
                # Print information for the found ROI
                #print(f"ROI Name: {roi_name}")
                #print(f"ROI NameTo find: {roi_name_to_find}")
                #print(f"ROI Number: {roi_number}")
                referenced_frame_of_reference_uid = roi_item.ReferencedFrameOfReferenceUID
                #print(f"Referenced Frame of Reference UID: {referenced_frame_of_reference_uid}")
                return referenced_frame_of_reference_uid

        else:
            return None


In [5]:
def FindUIDMatch(RtPaths,CTPaths,label):
    numcontours=0
    for currRTpath in RTStruct_fpaths:
        for currCTpath in CTPaths:
            #for label in labels:
            ctUID = FindCT_UID(currCTpath)
            rtUID = FindRT_UID_perContour(currRTpath,label)
            if rtUID == ctUID and ctUID is not None and rtUID is not None:
                numcontours+=1                    
                print("Yes Match",label)
                break
                break
    return numcontours

In [6]:
def GetTheContourRTMatch(ctfolder,rtfolder,tot_labels,tumor_labels,ln_labels):
    singlematch = False
    ct_Match = []
    tot_Match, tumor_Match, ln_Match = ([],[],[])
    if len(ctfolder)==0:
        print("No CT")
    for k in range(len(ctfolder)):
        for j in range(len(rtfolder)):
            rtfiles = os.listdir(rtfolder[j])
            for i in range(len(rtfiles)):
                if not singlematch:
                    try:
                        print("check",i,j,k)
                        #rtstruct = RTStructBuilder.create_from(dicom_series_path=ctfolder[k],rt_struct_path=str(rtfolder[j]+rtfiles[i]))
                        for labeltot in tot_labels:
                            numITVs = FindUIDMatch(RTStruct_fpaths,AvgCt_fpaths,labeltot)
                            if numITVs>0:
                                tot_Match.append(str(rtfolder[j]+rtfiles[i]))
                                singlematch = True
                                break
                        for labeltum in tumor_labels:
                            numITVs = FindUIDMatch(RTStruct_fpaths,AvgCt_fpaths,labeltum)
                            if numITVs>0:
                                tumor_Match.append(str(rtfolder[j]+rtfiles[i]))
                                singlematch = True
                                break
                        for labelln in ln_labels:
                            numITVs = FindUIDMatch(RTStruct_fpaths,AvgCt_fpaths,labelln)
                            if numITVs>0:
                                ln_Match.append(str(rtfolder[j]+rtfiles[i]))

                        if len(tot_Match)+len(tumor_Match)+len(ln_Match) == 0:
                            print("No labels Matched: ")
                        else:
                            ct_Match.append(ctfolder[k])
                            if len(tot_Match) == 0: tot_Match.append(0)
                            if len(tumor_Match) == 0: tumor_Match.append(0)
                            if len(ln_Match) == 0: ln_Match.append(0)
                    except Exception as e: pass

        #mask_3d = rtstruct.get_roi_mask_by_name(label)
    return ct_Match,tot_Match,tumor_Match,ln_Match

In [7]:
#GET PATHS AND LABELS
#root_path  = "//zkh/appdata/RTDicom/Projectline - Modelling lung cancer outcomes [panama code]/DICOM_data/DATA_VOLLEDIG_unstructured/"
root_path  = "//zkh/appdata/RTDicom/DAMEproject/LungSABR_data/Dicom data renamed/"
PxList = np.sort(os.listdir(root_path))
#csv_file_path = "//zkh/appdata/RTDicom/Projectline - Modelling lung cancer outcomes [panama code]/Users/Luis/CSVs/CTandRTMatch.csv"
csv_file_path = "//zkh/appdata/RTDicom/DAMEproject/LungSABR_data/CSV - Copy/LUIS_CSV/CTandRTMatch.csv"
#ITV LABELS
tot_labels = ['ITV',"IGTV","IgTV","ITV1","ITV2","ITV3","ITV totaal def","2ITV","ITV_TOT","ITV_6000","ITV_5100","ITV_Totaal","ITV_LBK","ITV_LOK"]
tumor_labels = ['ITVtumor',"ITV_tumor","ITVtumor def","2ITV_tumor","ITV-P","ITVtumorA1","ITV_tumor_LBK"]
ln_labels = ['ITVklieren',"ITV_klier","ITV_Klier","ITVklieren def","2ITV_klier","ITV_n","ITV_klier_LBK"]
fieldnames = ['ID', 'CT', 'Tot','Tumor',"LymphNodes"]
#GTV LABELS
gtv_tot_labels = ['GTV',"GTV1","GTV2","GTV3","GTV totaal def","2GTV","GTV_TOT","GTV_6000","GTV_5100","GTV_Totaal","GTV_LBK","GTV_LOK"]
gtv_tumor_labels = ['GTVtumor',"GTV_tumor","GTVtumor def","2GTV_tumor","GTV-P","GTVtumorA1","GTV_tumor_LBK"]
gtv_ln_labels = []
fieldnames = ['ID', 'CT', 'Tot','Tumor',"LymphNodes"]




In [8]:
#LOOK for patients already been analized
if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
    checked_list = df['ID'].tolist()
    print(len(checked_list),"Px already checked,out of",len(PxList))
    

34 Px already checked,out of 350


In [ ]:
with open(csv_file_path, 'a', newline='') as csvfile:
    
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    if csvfile.tell() == 0:
        writer.writeheader()

    for Px in PxList:
        AvgCt_fpaths = []
        RTStruct_fpaths = []
        if not os.path.exists(csv_file_path):
            checked_list = []
        if int(Px) in checked_list:
            print("Px already checked")
        else:
            for root, dirs, files in os.walk(root_path+Px, topdown=True):
                for d in dirs:
                    if "%" in d or "Thorax" in d or "Ave" in d or "MIP" in d or ("CT" in d and not "RTSTRUCT" in d):
                        AvgCt_fpaths.append(root+'/'+d+'/')
                    if ('RTSTRUCT' in d or "pproved" in d) or ("PinnPlan" in d and not("RTDOSE" in d)):
                        RTStruct_fpaths.append(root+'/'+d+'/')


            print("Px",Px,"Num CTs",len(AvgCt_fpaths),"Num RTs",len(RTStruct_fpaths))

            ctMatch,rtMatch_tot,rtMatch_tumor,rtMatch_ln = GetTheContourRTMatch(AvgCt_fpaths,RTStruct_fpaths,gtv_tot_labels,gtv_tumor_labels,gtv_ln_labels)

            if ctMatch ==None:
                print("No Match")
            else: 
                print("Yes Match")
                writer.writerow({'ID':Px,'CT':ctMatch,'Tot':rtMatch_tot,'Tumor':rtMatch_tumor,'LymphNodes':rtMatch_ln})





Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px already checked
Px 0694966 Num CTs 11 Num RTs 2
check 0 0 0
Yes Match GTV
check 0 1 0
Yes Match GTV
check 0 0 1
Yes Match GTV
check 0 1 1
Yes Match GTV
check 0 0 2
Yes Match GTV
check 0 1 2
Yes Match GTV
check 0 0 3
Yes Match GTV
check 0 1 3
Yes Match GTV
check 0 0 4
Yes Match GTV
check 0 1 4
Yes Match GTV
check 0 0 5
Yes Match GTV
check 0 1 5
Yes Match GTV
check 0 0 

Yes Match GTV
check 0 1 1
Yes Match GTV
Yes Match GTV
check 0 2 1
Yes Match GTV
Yes Match GTV
check 1 2 1
Yes Match GTV
Yes Match GTV
check 2 2 1
Yes Match GTV
Yes Match GTV
check 3 2 1
Yes Match GTV
Yes Match GTV
check 4 2 1
Yes Match GTV
Yes Match GTV
check 5 2 1
Yes Match GTV
Yes Match GTV
check 6 2 1
Yes Match GTV
Yes Match GTV
check 0 0 2
Yes Match GTV
Yes Match GTV
check 0 1 2
Yes Match GTV
Yes Match GTV
check 0 2 2
Yes Match GTV
Yes Match GTV
check 1 2 2
Yes Match GTV
Yes Match GTV
check 2 2 2
Yes Match GTV
Yes Match GTV
check 3 2 2
Yes Match GTV
Yes Match GTV
check 4 2 2
Yes Match GTV
Yes Match GTV
check 5 2 2
Yes Match GTV
Yes Match GTV
check 6 2 2
Yes Match GTV
Yes Match GTV
check 0 0 3
Yes Match GTV
Yes Match GTV
check 0 1 3
Yes Match GTV
Yes Match GTV
check 0 2 3
Yes Match GTV
Yes Match GTV
check 1 2 3
Yes Match GTV
Yes Match GTV
check 2 2 3
Yes Match GTV
Yes Match GTV
check 3 2 3
Yes Match GTV
Yes Match GTV
check 4 2 3
Yes Match GTV
Yes Match GTV
check 5 2 3
Yes Match GTV


Yes Match GTV
Yes Match GTV
Yes Match
Px 2069995 Num CTs 15 Num RTs 2
check 0 0 0
Yes Match GTV
Yes Match
Px 2086904 Num CTs 11 Num RTs 3
check 0 0 0
Yes Match GTV
Yes Match GTV
Yes Match GTV
Yes Match
Px 2091775 Num CTs 11 Num RTs 3
check 0 0 0
Yes Match GTV
Yes Match GTV
Yes Match
Px 2095366 Num CTs 11 Num RTs 2
check 0 0 0
Yes Match GTV
Yes Match GTV
Yes Match
Px 2097101 Num CTs 11 Num RTs 2
check 0 0 0
Yes Match GTV
Yes Match GTV
Yes Match
Px 2097887 Num CTs 11 Num RTs 3
check 0 0 0
Yes Match GTV
Yes Match GTV
Yes Match
Px 2110537 Num CTs 11 Num RTs 2
check 0 0 0
Yes Match GTV
Yes Match GTV
Yes Match
Px 2141762 Num CTs 1 Num RTs 1
check 0 0 0
Yes Match GTV
Yes Match
Px 2177148 Num CTs 5 Num RTs 2
check 0 0 0
Yes Match GTV
Yes Match
Px 2191764 Num CTs 9 Num RTs 2
check 0 0 0
Yes Match GTV
Yes Match GTVtumor
Yes Match
Px 2211762 Num CTs 11 Num RTs 3
check 0 0 0
Yes Match GTV
Yes Match GTV
Yes Match GTV
Yes Match
Px 2235939 Num CTs 16 Num RTs 2
check 0 0 0
Yes Match GTV
Yes Match
Px 2

Yes Match GTV
check 0 1 7
Yes Match GTV
check 0 0 8
Yes Match GTV
check 0 1 8
Yes Match GTV
check 0 0 9
Yes Match GTV
check 0 1 9
Yes Match GTV
check 0 0 10
Yes Match GTV
check 0 1 10
Yes Match GTV
Yes Match
Px 3707491 Num CTs 1 Num RTs 1
check 0 0 0
Yes Match GTV
Yes Match
Px 3716437 Num CTs 11 Num RTs 2
check 0 0 0
Yes Match GTV
Yes Match GTV
Yes Match
Px 3731300 Num CTs 1 Num RTs 1
check 0 0 0
Yes Match GTV
Yes Match
Px 3742605 Num CTs 1 Num RTs 1
check 0 0 0
Yes Match GTV
Yes Match
Px 3747620 Num CTs 11 Num RTs 2
check 0 0 0
Yes Match GTV_6000
Yes Match GTV_6000
Yes Match
Px 3766412 Num CTs 11 Num RTs 2
check 0 0 0
Yes Match GTV
Yes Match GTV
Yes Match
Px 3791088 Num CTs 15 Num RTs 2
check 0 0 0
Yes Match GTV
Yes Match
Px 3831305 Num CTs 12 Num RTs 2
check 0 0 0
Yes Match GTV1
Yes Match
Px 3892451 Num CTs 11 Num RTs 2
check 0 0 0
Yes Match GTV
Yes Match GTV
Yes Match
Px 3927184 Num CTs 12 Num RTs 2
check 0 0 0
Yes Match GTV
Yes Match
Px 3927622 Num CTs 12 Num RTs 2
check 0 0 0
Yes 